# PixelFrame engineering design demo

To do

Shear from steel fiber.

Import required packages

In [1]:
using Pkg
Pkg.activate("..");
Pkg.status()
Pkg.instantiate()

  Activating project at `~/dev/ConcreteExplorer`


Status `~/dev/ConcreteExplorer/Project.toml`
  [acfe5cae] AsapSections v0.1.0 `https://github.com/keithjlee/AsapSections#main`
  [336ed68f] CSV v0.10.12
  [13f3f980] CairoMakie v0.11.5
  [a93c6f00] DataFrames v1.6.1
  [59287772] Formatting v0.4.2
  [f6369f11] ForwardDiff v0.10.36
  [e9467ef8] GLMakie v0.9.5
  [682c06a0] JSON v0.21.4
  [ee78f7c6] Makie v0.20.4
  [01bcebdf] Nonconvex v2.1.3
  [d3d89cbb] NonconvexMMA v1.0.0
  [b43a31b8] NonconvexNLopt v0.1.8
  [429524aa] Optim v1.7.8
⌃ [7f7a1694] Optimization v3.20.2
⌃ [4e6fcdb7] OptimizationNLopt v0.1.8
  [90137ffa] StaticArrays v1.9.1
  [e88e6eb3] Zygote v0.6.68
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [48]:
using CSV
using DataFrames
using JSON
using Dates
using ProgressBars

Load the precalculated catalog

In [4]:
pwd()

"/Users/pitipatwongsittikan/dev/ConcreteExplorer/src"

In [31]:
catalog = CSV.read("Catalogs/test_catalog.csv", DataFrame); #load the pre-calc catalog
sort!(catalog, [:carbon, :fc′, :as, :ec])


println("The catalog was sorted by ascending order from:\ncarbon -> fc′ -> as -> ec")
println(catalog[1:20,:])

The catalog was sorted by ascending order from:
carbon -> fc′ -> as -> ec
20×9 DataFrame
 Row │ fc′      as       ec       fpe      Pu       Mu       Vu       carbon   ID    
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Int64 
─────┼───────────────────────────────────────────────────────────────────────────────
   1 │    28.0     99.0     0.05      0.0  627.772  109.659  8.14936  16.6541      1
   2 │    28.0     99.0     0.05     46.5  625.378  108.641  8.14928  16.6541      2
   3 │    28.0     99.0     0.05     93.0  622.985  107.706  8.1492   16.6541      3
   4 │    28.0     99.0     0.05    139.5  620.591  106.846  8.14912  16.6541      4
   5 │    28.0     99.0     0.05    186.0  618.197  106.032  8.14904  16.6541      5
   6 │    28.0     99.0     0.05    232.5  615.803  108.382  8.14896  16.6541      6
   7 │    28.0     99.0     0.05    279.0  613.409  110.748  8.14887  16.6541      7
   8 │    28.0     99.0     0.05    325.5  611.016  113.11

Load the demand file   

In [41]:
#load demands into a dictionary
demand_path = joinpath(@__DIR__, "Demands/test_input_CISBAT_dataset.json");
open(demand_path, "r") do f
    global demands = DataFrame(JSON.parse(f, dicttype=Dict{String,Any}))
    ns = size(demands)[1]
    demands[!,:idx] = 1:ns
    println("Demands were loads from:\n", demand_path)
end
println(demands[1:10,:])

Demands were loads from:
/Users/pitipatwongsittikan/dev/ConcreteExplorer/src/Demands/test_input_CISBAT_dataset.json
10×8 DataFrame
 Row │ vu        s_idx  pu            e_idx  type     ec_max     mu       idx   
     │ Float64   Int64  Float64       Int64  String   Float64    Float64  Int64 
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 129.755       0  -1.4779e-15       0  primary  0.0423597  129.755      1
   2 │ 129.755       1  -7.38948e-16      0  primary  0.117666   194.632      2
   3 │ 129.755       2  -7.38948e-16      0  primary  0.180421   259.509      3
   4 │  45.9809      3   1.4779e-15       0  primary  0.230625   283.246      4
   5 │  45.9809      4   7.38948e-16      0  primary  0.268278   306.237      5
   6 │  45.9809      5  -7.38948e-16      0  primary  0.29338    329.227      6
   7 │  45.9809      6  -7.38948e-16      0  primary  0.305931   352.218      7
   8 │  45.9809      7   0.0              0  primary  0.305931   3

The demand catalog was labeled with 0 index scheme, add 1 so they are compatible with Julia 1 index scheme.

In [42]:
@show old_min_e_idx = minimum(demands[!, "e_idx"]);
@show old_min_s_idx = minimum(demands[!, "s_idx"]);

@show old_max_e_idx = maximum(demands[!, "e_idx"]);
@show old_max_s_idx = maximum(demands[!, "s_idx"]);



old_min_e_idx = minimum(demands[!, "e_idx"]) = 0
old_min_s_idx = minimum(demands[!, "s_idx"]) = 0
old_max_e_idx = maximum(demands[!, "e_idx"]) = 29
old_max_s_idx = maximum(demands[!, "s_idx"]) = 13


In [43]:
demands[!,"e_idx"] .+= 1 ;
demands[!,"s_idx"] .+= 1 ;

In [44]:
@show new_min_e_idx = minimum(demands[!, "e_idx"]);
@show new_min_s_idx = minimum(demands[!, "s_idx"]);

@show new_max_e_idx = maximum(demands[!, "e_idx"]);
@show new_max_s_idx = maximum(demands[!, "s_idx"]);


new_min_e_idx = minimum(demands[!, "e_idx"]) = 1
new_min_s_idx = minimum(demands[!, "s_idx"]) = 1
new_max_e_idx = maximum(demands[!, "e_idx"]) = 30
new_max_s_idx = maximum(demands[!, "s_idx"]) = 14


Fix redundant element indices due to Primary and Secondary labels

In [45]:
global e_idx = 1 
for i in 1:size(demands)[1]
    if i !=1
        demands[i-1, :e_idx] = e_idx
        if demands[i,:s_idx] < demands[i-1,:s_idx]
            global e_idx +=1 
        end
        
    end
    if i == size(demands)[1]
        demands[i, :e_idx] = e_idx
    end
end

Create a function to filter only feasible designs in a demand point

In [46]:

"""
filter feasible demands
"""
function filter_demands(demands::DataFrame, catalog::DataFrame)

    ns = size(demands)[1]          #total number of sections
    ne = unique(demands[!, :e_idx]) #total number of elements
    nc = size(catalog, 1)           #total number of available choices

    output_results = Dict{Int64,Vector{Int64}}() #map between demands and indices of the feasible section.
    demands[!, "total_results"] = zeros(Int64, size(demands)[1])
    #go through each section and filter the feasible designs from the catalog.
    for i = 1:ns
        en = demands[i, "e_idx"]
        sn = demands[i, "s_idx"]
        # push!(elements_to_sections[en], sn)

        pu = demands[i, "pu"]
        mu = demands[i, "mu"]
        # vu = demands[i,"vu"]
        ec_max = demands[i, "ec_max"]

        global feasible_sections = filter([:Pu, :Mu, :Vu, :ec] => (x1, x2, x3, x4) ->
                x1 > pu &&
                x2 > mu &&
                # x3 > vu && Have not consider the shear
                x4 <= ec_max * 1000,
            catalog
        )

        if size(feasible_sections)[1] == 0 #if the number of feasible results = 0
            println(feasible_sections[!, :ID])
            println("section $sn: element $en")
            output_results[i] = [0]
            demands[i, "total_results"] = 0
            # println(outr)
        else
            output_results[i] = feasible_sections[!, :ID]
            demands[i, "total_results"] = length(feasible_sections[!, :ID])
            # println(outr)
        end
    end

    return output_results
end


filter_demands

In [63]:
"""
Find the optimum result for each element.
    
    !! not optimum
For the same element, will use the same fc′ steel size and post tensioning stress.

"""
function find_optimum(output_results, demands)
    total_ns = size(demands)[1] #get total number of section points.
    ne = unique(demands[!, :e_idx]) #number of elements

    #Map element indices to indices of sections indices on those elements.
    elements_to_sections = Dict(k => Int[] for k in unique(demands[!, "e_idx"]))
    for i = 1:total_ns
        en = demands[i, "e_idx"]
        # sn = demands[i, "s_idx"]
        push!(elements_to_sections[en], i)
    end


    #element index to list of designs
    elements_designs = Dict(k => [[]] for k in unique(demands[!, "e_idx"])) 


    #find fc′, as, and fpe that appear in all sections in an element.
    for i in ProgressBar(ne) #loop each element
        println("Element $i out of $(length(ne)) elements")
        sections = elements_to_sections[i] #sections associated with this element

        ns = length(sections)     # number of sections in this element 

        #start from the middle-ish section (n/2 or (n-1)/2)
        #note that section is in the form of 1,2,3,..., ns.

        mid = div(ns, 2)
        
        #get the feasible designs for the middle section
        feasible_idx = output_results[sections[mid]]

        #catalog was already sorted, so I think we can leave this part, just filter, to save time.
        mid_catalog = sort(catalog[feasible_idx, :], [:carbon, :fc′, :ec])
        # mid_catalog = catalog[feasible_idx, :]

        #now, loop each design in the sub catalog, see if as and fpe are available in all sections.
        #if not, remove that design from the sub catalog.
        #if yes, keep it.

        #select each design, check if as and fpe exist for the the section
        global_d = 0
        for d_idx in 1:size(catalog)[1]
            d = mid_catalog[d_idx, :]

            all_as = true
            all_fpe = true
            for s in sections
                #check if the design is available in that section.
                #if not, remove it from the sub catalog.
                #if yes, keep it.
                # if !(d[:fc′] ∈ catalog[output_results[s], :fc′])
                #     all_fc′ = false
                # end
                if !(d[:as] ∈ catalog[output_results[s], :as])
                    all_as = false
                    break
                end
                if !(d[:fpe] ∈ catalog[output_results[s], :fpe])
                    all_fpe = false
                    break
                end
            end
            if all_as && all_fpe
                #first run, found, move on.
                global_d = d_idx
                break
            end
        end

        #get the first one, they will appear in the entire thing anyway.
        this_fpe = mid_catalog[global_d, :fpe]
        this_as = mid_catalog[global_d, :as]

        sections_designs = Vector{Vector}(undef, ns)
        for is in eachindex(elements_to_sections[i])
            #current section index
            s = elements_to_sections[i][is]

            feasible_idx = output_results[s]
            
            fpe_as(fpe::Float64, as::Float64) = fpe == this_fpe && as == this_as

            this_catalog = filter([:fpe, :as] => fpe_as, catalog[output_results[s], :])

            sort!(this_catalog, [:carbon, :ec])

            #get the first one, it's the lowest carbon
            select_ID = this_catalog[1, :ID]
            #find lowest e for this one.
            sections_designs[is] = collect(catalog[select_ID, :])
            # println(sections_designs[is])
        end

        elements_designs[i] = sections_designs

    end

    return elements_designs
end

find_optimum

In [66]:
output_results = filter_demands(demands,catalog)


Dict{Int64, Vector{Int64}} with 552 entries:
  56  => [16, 17, 18, 19, 20, 21, 22, 23, 24, 25  …  21799, 21800, 21801, 21802…
  35  => [975, 976, 977, 978, 979, 1003, 1004, 1005, 1006, 1007  …  21799, 2180…
  425 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  429 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  60  => [6658, 6659, 6660, 6661, 6686, 6687, 6688, 6689, 6690, 6691  …  21799,…
  220 => [134, 135, 136, 160, 161, 162, 163, 164, 165, 166  …  21799, 21800, 21…
  308 => [45, 46, 47, 48, 49, 50, 51, 52, 53, 54  …  21799, 21800, 21801, 21802…
  67  => [6658, 6659, 6660, 6661, 6686, 6687, 6688, 6689, 6690, 6691  …  21799,…
  215 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  21799, 21800, 21801, 21802, 21803, …
  73  => [569, 570, 571, 572, 573, 596, 597, 598, 599, 600  …  21799, 21800, 21…
  319 => [45, 46, 47, 48, 49, 50, 51, 52, 53, 54  …  21799, 21800, 21801, 21802…
  251 => [132, 133, 134, 135, 136, 137, 138, 139, 140, 141  …  2

In [67]:
elements_designs = find_optimum(output_results, demands)


Element 1 out of 57 elements
Real[28.0, 99.0, 0.05, 790.5, 587.0773272445823, 135.8586102792226, 8.147986076036542, 16.65411779534618, 18]
Real[28.0, 99.0, 0.275, 790.5, 587.0773272445823, 196.97586225856293, 8.147986076036542, 16.65411779534618, 279]
Real[28.0, 99.0, 0.525, 790.5, 587.0773272445823, 264.88392001338553, 8.147986076036542, 16.65411779534618, 569]
Real[28.0, 99.0, 0.6, 790.5, 587.0773272445823, 285.25633733983227, 8.147986076036542, 16.65411779534618, 656]
Real[28.0, 99.0, 0.7, 790.5, 587.0773272445823, 312.4195604417613, 8.147986076036542, 16.65411779534618, 772]
Real[28.0, 99.0, 0.775, 790.5, 587.0773272445823, 332.7919777682081, 8.147986076036542, 16.65411779534618, 859]
Real[28.0, 99.0, 0.85, 790.5, 587.0773272445823, 353.1643950946549, 8.147986076036542, 16.65411779534618, 946]
Real[28.0, 99.0, 0.85, 790.5, 587.0773272445823, 353.1643950946549, 8.147986076036542, 16.65411779534618, 946]
Real[28.0, 99.0, 0.775, 790.5, 587.0773272445823, 332.7919777682081, 8.147986076

0.0%┣                                               ┫ 0/57 [00:00<00:00, -0s/it]


[28.0, 99.0, 0.3, 790.5, 587.0773272445823, 203.76666803404518, 8.147986076036542, 16.65411779534618, 308]
Real[28.0, 99.0, 0.525, 790.5, 587.0773272445823, 264.88392001338553, 8.147986076036542, 16.65411779534618, 569]
Real[28.0, 99.0, 0.625, 790.5, 587.0773272445823, 292.04714311531455, 8.147986076036542, 16.65411779534618, 685]
Real[28.0, 99.0, 0.7, 790.5, 587.0773272445823, 312.4195604417613, 8.147986076036542, 16.65411779534618, 772]
Real[28.0, 99.0, 0.8, 790.5, 587.0773272445823, 339.58278354369037, 8.147986076036542, 16.65411779534618, 888]
Real[28.0, 99.0, 0.875, 790.5, 587.0773272445823, 359.95520087013716, 8.147986076036542, 16.65411779534618, 975]
Real[28.0, 99.0, 0.875, 790.5, 587.0773272445823, 359.95520087013716, 8.147986076036542, 16.65411779534618, 975]
Real[28.0, 99.0, 0.8, 790.5, 587.0773272445823, 339.58278354369037, 8.147986076036542, 16.65411779534618, 888]
Real[28.0, 99.0, 0.7, 790.5, 587.0773272445823, 312.4195604417613, 8.147986076036542, 16.65411779534618, 772]

, 9529]
Real[40.0, 99.0, 1.2, 744.0, 858.5164046351176, 647.08673993204, 8.771534138140208, 20.151358481562895, 9529]
Real[40.0, 99.0, 1.1, 744.0, 858.5164046351176, 608.2821355007128, 8.771534138140208, 20.151358481562895, 9413]
Real[36.0, 99.0, 1.175, 744.0, 768.8346521716056, 571.8544794360441, 8.580677697366024, 19.087377152993305, 6774]
Real[36.0, 99.0, 1.05, 744.0, 768.8346521716056, 528.2305361936585, 8.580677697366024, 19.087377152993305, 6629]
Real[32.0, 99.0, 1.15, 744.0, 679.1528997080941, 494.31750741810816, 8.374076236102818, 17.921630257587733, 4019]
Real[28.0, 99.0, 0.925, 744.0, 589.4711472445822, 366.8786685313988, 8.148066867461543, 16.65411779534618, 1032]
Real[28.0, 99.0, 0.475, 744.0, 589.4711472445822, 247.05124657982915, 8.148066867461543, 16.65411779534618, 510]
Element 10 out of 57 elements
Real[28.0, 99.0, 0.05, 790.5, 587.0773272445823, 135.8586102792226, 8.147986076036542, 16.65411779534618, 18]
Real[28.0, 99.0, 0.05, 790.5, 587.0773272445823, 135.8586102792

8.8%┣████▏                                          ┫ 5/57 [00:00<00:01, 58it/s]
19.3%┣████████▊                                    ┫ 11/57 [00:00<00:01, 80it/s]


...

 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Element 49 out of 57 elements
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Element 50 out of 57 elements
Real[28.0, 99.0, 0.05, 0.0, 627.77

64.9%┣████████████████████████████▋               ┫ 37/57 [00:00<00:00, 104it/s]
82.5%┣████████████████████████████████████▎       ┫ 47/57 [00:00<00:00, 115it/s]


Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Element 55 out of 57 elements
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1]
Real[28.0, 99.0, 0.05, 0.0, 627.7722672445823, 109.65913396855397, 8.149359530261542, 16.65411779534618, 1

100.0%┣███████████████████████████████████████████┫ 57/57 [00:00<00:00, 125it/s]
100.0%┣███████████████████████████████████████████┫ 57/57 [00:00<00:00, 124it/s]


Dict{Int64, Vector{Vector{Any}}} with 57 entries:
  5  => [[28.0, 99.0, 0.5, 697.5, 591.865, 248.836, 8.14815, 16.6541, 538], [28…
  56 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  16 => [[28.0, 99.0, 0.05, 790.5, 587.077, 135.859, 8.14799, 16.6541, 18], [28…
  20 => [[28.0, 99.0, 0.05, 697.5, 591.865, 131.64, 8.14815, 16.6541, 16], [28.…
  35 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  55 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  30 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  19 => [[28.0, 99.0, 0.05, 790.5, 587.077, 135.859, 8.14799, 16.6541, 18], [28…
  32 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  49 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936, 16.6541, 1], [28.0,…
  6  => [[28.0, 99.0, 0.05, 790.5, 587.077, 135.859, 8.14799, 16.6541, 18], [28…
  45 => [[28.0, 99.0, 0.05, 0.0, 627.772, 109.659, 8.14936,

Save the design result

In [68]:
open("Results/designs_results_13_01.json","w") do f
    JSON.print(f, elements_designs)
end

## 3. Visualizing the results